In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 反馈或问题？
如有任何反馈或问题，请打开一个问题。

用于Python的顶点SDK：自定义训练和AutoML表格训练示例

要使用这个Colaboratory笔记本，您可以将笔记本复制到自己的谷歌云盘并在Colaboratory（或Colab）中打开它。您可以运行每个步骤或单元格，并查看其结果。要运行一个单元格，请使用Shift+Enter。Colab会自动显示每个单元格中最后一行的返回值。有关在Colab中运行笔记本的更多信息，请参阅[Colab欢迎页面](https://colab.research.google.com/notebooks/welcome.ipynb)。

本笔记本演示了如何基于表格数据集创建AutoML模型和自定义模型。您需要提供一个存储数据集的存储桶。

注意：在测试这个SDK时，您可能会因为训练、预测、存储或使用其他GCP产品而产生费用。

# 安装用于Python的Vertex SDK，进行身份验证，并将数据集上传至您的GCS存储桶

安装SDK后，内核将自动重新启动。您可能会看到错误消息 `您的会话由于未知原因而崩溃`，这是正常的。

In [ ]:
!pip3 uninstall -y google-cloud-aiplatform
!pip3 install google-cloud-aiplatform
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
import sys

if "google.colab" in sys.modules:
    import os

    from google.colab import auth

    auth.authenticate_user()
    os.environ["GOOGLE_CLOUD_PROJECT"] = MY_PROJECT

### 输入您的项目ID和GCS存储桶

在下方单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在此笔记本中的所有命令中使用正确的项目。

In [ ]:
MY_PROJECT = "YOUR PROJECT"
MY_STAGING_BUCKET = "gs://YOUR BUCKET"  # bucket should be in same region as Vertex AI

我们使用的数据集是加州住房数据集，可在Colab本地使用。有关此数据集及其许可的更多信息，请访问: https://developers.google.com/machine-learning/crash-course/california-housing-data-description

In [ ]:
!gcloud config set project {MY_PROJECT}

TRAIN_FILE_NAME = "california_housing_train.csv"
!gsutil cp sample_data/{TRAIN_FILE_NAME} {MY_STAGING_BUCKET}/data/

gcs_csv_path = f"{MY_STAGING_BUCKET}/data/{TRAIN_FILE_NAME}"

初始化 Python 的 Vertex SDK

初始化 Vertex AI 的 *client*

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=MY_PROJECT, staging_bucket=MY_STAGING_BUCKET)

# 从CSV文件创建受控制的表格数据集

In [ ]:
ds = aiplatform.TabularDataset.create(
    display_name="housing", gcs_source=[gcs_csv_path], sync=False
)

# 编写您的训练脚本
- 将这个单元格写成一个文件，用于自定义训练。

In [ ]:
%%writefile training_script.py

import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


# uncomment and bump up replica_count for distributed training
# strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
# tf.distribute.experimental_set_strategy(strategy)

target = 'median_house_value'

def aip_data_to_dataframe(wild_card_path):
    return pd.concat([pd.read_csv(fp.numpy().decode())
                      for fp in tf.data.Dataset.list_files([wild_card_path])])

def get_features_and_labels(df):
    features = df.drop(target, axis=1) 
    return {key: features[key].values for key in features.columns}, df[target].values

def data_prep(wild_card_path):
    return get_features_and_labels(aip_data_to_dataframe(wild_card_path))

train_features, train_labels = data_prep(os.environ["AIP_TRAINING_DATA_URI"])

feature_columns = [tf.feature_column.numeric_column(name) for name in 
                   train_features.keys()]

model = tf.keras.Sequential([
            layers.DenseFeatures(feature_columns),
            layers.Dense(64),
            layers.Dense(1)])
model.compile(loss='mse', optimizer='adam')

model.fit(train_features, train_labels,
          epochs=10,
          validation_data=data_prep(os.environ["AIP_VALIDATION_DATA_URI"]))
print(model.evaluate(*data_prep(os.environ["AIP_TEST_DATA_URI"])))

# save as Vertex AI Managed model
tf.saved_model.save(model, os.environ["AIP_MODEL_DIR"])

启动训练AutoML和自定义训练作业以创建模型

一旦我们已经创建了您的数据集并定义了您的训练脚本，我们将创建一个模型。

In [ ]:
custom_job = aiplatform.CustomTrainingJob(
    display_name="train-housing",
    script_path="training_script.py",
    container_uri="gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest",
    requirements=[
        "gcsfs==0.7.1",
    ],
    model_serving_container_image_uri="gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest",
)

custom_model = custom_job.run(
    ds, replica_count=1, model_display_name="housing-model", sync=False
)

automl_job = aiplatform.AutoMLTabularTrainingJob(
    display_name="train-housing-automl_1",
    optimization_prediction_type="regression",
    optimization_objective="minimize-rmse",
    column_transformations=[
        {"numeric": {"column_name": "longitude"}},
        {"numeric": {"column_name": "latitude"}},
        {"numeric": {"column_name": "housing_median_age"}},
        {"numeric": {"column_name": "total_rooms"}},
        {"numeric": {"column_name": "total_bedrooms"}},
        {"numeric": {"column_name": "population"}},
        {"numeric": {"column_name": "households"}},
        {"numeric": {"column_name": "median_income"}},
    ],
    optimization_objective_recall_value=None,
    optimization_objective_precision_value=None,
)

# This will take around an hour to run
automl_model = automl_job.run(
    dataset=ds,
    target_column="median_house_value",
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    weight_column=None,
    budget_milli_node_hours=1000,
    model_display_name="house-value-prediction-model",
    disable_early_stopping=False,
    predefined_split_column_name=None,
    sync=False,
)

部署模型

In [ ]:
custom_endpoint = custom_model.deploy(machine_type="n1-standard-4", sync=False)
automl_endpoint = automl_model.deploy(machine_type="n1-standard-4", sync=False)

# 预测端点

In [ ]:
custom_endpoint.wait()

In [ ]:
# This sample is taken from an observation where median_house_value = 94600
prediction = custom_endpoint.predict(
    [
        {
            "longitude": -124.35,
            "latitude": 40.54,
            "housing_median_age": 52.0,
            "total_rooms": 1820.0,
            "total_bedrooms": 300.0,
            "population": 806,
            "households": 270.0,
            "median_income": 3.014700,
        },
    ]
)
prediction

In [ ]:
automl_job.state

In [ ]:
automl_endpoint.wait()

In [ ]:
# This sample is taken from an observation where median_house_value = 94600
prediction = automl_endpoint.predict(
    [
        {
            "longitude": "-124.35",
            "latitude": "40.54",
            "housing_median_age": "52.0",
            "total_rooms": "1820.0",
            "total_bedrooms": "300.0",
            "population": "806",
            "households": "270.0",
            "median_income": "3.014700",
        },
    ]
)
prediction